#semi-guided


In [1]:
import pandas as pd
import torch
from bertopic import BERTopic

torch.set_num_threads(16)

In [2]:
grouped = pd.read_pickle('../data/large-data/grouped_df.pkl')

In [3]:
annotations = pd.read_csv('../data/enemy-annotations - sentences.csv')

# Take only the first value if there are multiple categories
annotations['manual_label'] = annotations['polemical category'].astype(str).str.split(',').str[0].str.strip()

# Merge with grouped on 'sentence_id'
grouped = grouped.merge(
    annotations[['sentence_id', 'manual_label']],
    on='sentence_id',
    how='left'
)

import numpy as np

grouped['manual_label'] = grouped['manual_label'].replace('nan', np.nan)
grouped['manual_label'] = grouped['manual_label'].fillna(-1).astype(int)

In [4]:
# Split by 'lagt_provenience'
grouped_christian = grouped[grouped['lagt_provenience'] == 'christian']
grouped_pagan = grouped[grouped['lagt_provenience'] == 'pagan']

christian_0_300 = grouped_christian[
    (grouped_christian['not_before'] >= 0) & (grouped_christian['not_after'] <= 300)
]
christian_300_600 = grouped_christian[
    (grouped_christian['not_before'] >= 300) & (grouped_christian['not_after'] <= 600)
]

pagan_0_300 = grouped_pagan[
    (grouped_pagan['not_before'] >= 0) & (grouped_pagan['not_after'] <= 300)
]
pagan_300_600 = grouped_pagan[
    (grouped_pagan['not_before'] >= 300) & (grouped_pagan['not_after'] <= 600)
]

In [4]:
docs = grouped['lamma_sentence']
y = grouped['manual_label']

category_names = [
    "violence, troublemaking",
    "moral depravity",
    "idolatry, heresy, magic",
    "falseness, hypocrisy, inflated self-esteem",
    "evil/dubious agents",
    "general polemical term"
]

In [5]:
def run_bertopic_on_subcorpus(subcorpus):
    from bertopic import BERTopic
    from transformers.pipelines import pipeline

    docs = subcorpus['lamma_sentence']
    y = subcorpus['manual_label']

    category_names = [
        "violence, troublemaking",
        "moral depravity",
        "idolatry, heresy, magic",
        "falseness, hypocrisy, inflated self-esteem",
        "evil/dubious agents",
        "general polemical term"
    ]

    embedding_model = pipeline("feature-extraction", model="pranaydeeps/Ancient-Greek-BERT")
    topic_model = BERTopic(embedding_model=embedding_model,
                            n_gram_range = (1, 1),
                            verbose=True,
                            low_memory=True
    )

    topic_model.fit(docs, y=y)
    topic_model.save(f"../data/large-data/{subcorpus}_bertopic_semi_model", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)
    labeled_text_df = topic_model.get_document_info(docs)
    labeled_text_df.to_pickle(f"../data/large-data/{subcorpus}_bertopic_semi_labeled.pkl")


In [ ]:
run_bertopic_on_subcorpus(christian_0_300)
run_bertopic_on_subcorpus(christian_300_600)
run_bertopic_on_subcorpus(pagan_0_300)
run_bertopic_on_subcorpus(pagan_300_600)

In [3]:
import pandas as pd

semi = pd.read_pickle('../data/large-data/christian_0_300_bertopic_semi_labeled.pkl')
filtered = semi[semi['Topic'].isin(range(1, 8))]
filtered.shape

(6568, 8)

#guided

associated terms

Violence, trouble-making
στασιαστής (rebel), ταραχώδης (troublesome), φονεύς (murderer)
Moral depravity
πόρνος (fornicator), πλεονέκτης (greedy person), μέθυσος (drunkard)
Idolatry, heresy, magic
αἱρετικός (heretic), μάγος (sorcerer, magician)
Falseness, hypocrisy
ψεύστης (liar), ὑποκριτής (hypocrite)
Inflated self-esteem
ἀλαζών (boaster), φιλοδοξία (ambition, love of glory)
Evil/dubious agents
διάβολος (devil, slanderer), κακοποιός (evildoer, criminal), πονηρός (evil), ἀντίχριστος (Antichrist), Σατανᾶς (Satan)



#zero-shot

In [1]:
import pandas as pd
import torch
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from transformers.pipelines import pipeline

grouped = pd.read_pickle('../data/large-data/grouped_df.pkl')

category_names = [
    "violence, troublemaking",
    "moral depravity",
    "idolatry, heresy, magic",
    "falseness, hypocrisy, inflated self-esteem",
    "evil/dubious agents",
    "general polemical term"
]

torch.set_num_threads(16)

embedding_model = pipeline(
    "feature-extraction",
    model="pranaydeeps/Ancient-Greek-BERT",
    #device=-1  # Use CPU
)

from umap import UMAP
umap_model = UMAP(n_neighbors=8,
                  metric='cosine',
                  n_components=2,
                  random_state=42,
                    )


#Zero-shot mode
topic_model = BERTopic(verbose=True, 
                        embedding_model=embedding_model,
                        min_topic_size=15,
                        zeroshot_topic_list=category_names,
                        zeroshot_min_similarity=0,
                        representation_model=KeyBERTInspired(),
                        umap_model=umap_model,
                        )


topics, _  = topic_model.fit_transform(grouped['lamma_sentence'])
topic_model.save("../data/large-data/topic_model")

labeled_text_df = topic_model.get_document_info(grouped['lamma_sentence'])
labeled_text_df.to_pickle("../data/large-data/labeled_text_df.pkl")

Device set to use cuda:0
2025-08-04 14:23:07,699 - BERTopic - Embedding - Transforming documents to embeddings.
100%|██████████| 1778995/1778995 [1:53:34<00:00, 261.07it/s]  
2025-08-04 16:16:45,322 - BERTopic - Embedding - Completed ✓
2025-08-04 16:16:45,344 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


: 

In [ ]:
#load trained model
topic_model = BERTopic.load("../data/large-data/topic_model")
labeled_text_df.read_pickle("../data/large-data/labeled_text_df.pkl")

In [13]:
#print(f'{topic_model.get_topic_info()['Count'][0]/len(df)}% are unlabeled outliers.')
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,62071,-1_ὑπάρχω_δίδωμι_εὑρίσκω_μέλλω,"[ὑπάρχω, δίδωμι, εὑρίσκω, μέλλω, παῖς, χράω, φ...",[σκοπέω ἔννοια ἔρχομαι ἔοικα εἰμί θεός ἀνήρ πα...
1,0,2157,0_καταφρονέω_ἀγανακτέω_φρονέω_δίδωμι,"[καταφρονέω, ἀγανακτέω, φρονέω, δίδωμι, αἰτέω,...","[οὐδείς ἀδικέω μισέω, λέγω δοκέω ψυχή κινέω κί..."
2,1,2092,1_μεταβάλλω_μέλλω_φαίνω_διαβαίνω,"[μεταβάλλω, μέλλω, φαίνω, διαβαίνω, λευκός, δι...","[γίγνομαι ω, γίγνομαι γίγνομαι, γίγνομαι γίγνο..."
3,2,1113,2_προστακτικός_μιμητής_συλλογισμός_αἴτιος,"[προστακτικός, μιμητής, συλλογισμός, αἴτιος, ὑ...","[γίγνομαι β, ι̅ γίγνομαι ρ̅, γίγνομαι ε]"
4,3,752,3_δριμύς_ποσός_οἰκεῖος_πικρός,"[δριμύς, ποσός, οἰκεῖος, πικρός, δυκεῖος, ψῦξι...",[ὑγρός ξηρός μετέχω ἴσος θερμότης ψῦξις εἰμί μ...
...,...,...,...,...,...
568,567,15,567_ξένος_βύζω_τιμάω_πάππος,"[ξένος, βύζω, τιμάω, πάππος, χαίρω, πρόγονος, ...",[Ὀδυσσεύς ἑταῖρος Ἥλιος βοῦς ἐργάτης θοινάω ἀτ...
569,568,15,568_σκανδαλίζω_ὁδεύσος_ἀπόλλυμι_ἀντίλυτρος,"[σκανδαλίζω, ὁδεύσος, ἀπόλλυμι, ἀντίλυτρος, μέ...",[κρίνω Ἀργεῖος ἀποφεύγω Ἀφροδίτη ἀνατίθημι Νικ...
570,569,15,569_οἰκεῖος_φελλεύς_ἀγαθός_ἁμαρτωλός,"[οἰκεῖος, φελλεύς, ἀγαθός, ἁμαρτωλός, ἰδιώτης,...",[πρῶτος ἰδιώτης βασιλικός κόσμιος ἐρῆμος ἐπιφο...
571,570,15,570_ἔρχομαι_βασιλἠων_ἀξίωμα_βασιλείας,"[ἔρχομαι, βασιλἠων, ἀξίωμα, βασιλείας, χρυσοστ...","[βασιλεία, βασιλείας, βασιλείας]"
